# 1.训练集数据预处理

1.1导入数据，补空

In [7]:
import pandas as pd
import numpy as np
from math import isnan

qr=pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='迁入',index_col=[0,1])
qc=pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='迁出',index_col=[0,1])
qc_bili = pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='1.10-1.16武汉迁出比例',index_col=0)

qc_train = qc.iloc[:,:7]
qr_train = qr.iloc[:,:7]


In [10]:
qc_train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 369 entries, (湖北, 武汉) to (新疆, 昆玉)
Data columns (total 7 columns):
1.1     360 non-null float64
1.11    360 non-null float64
1.12    360 non-null float64
1.13    360 non-null float64
1.14    360 non-null float64
1.15    360 non-null float64
1.16    360 non-null float64
dtypes: float64(7)
memory usage: 24.6+ KB


1.2将每个空的值改为该省份的平均值

In [11]:

qr_train.loc['江西', '赣江新区']=qr_train.loc['江西'].mean()
qc_train.loc['江西', '赣江新区']=qc_train.loc['江西'].mean()

qr_train.loc['海南','三沙']=qr_train.loc['海南'].mean()
qc_train.loc['海南','三沙']=qc_train.loc['海南'].mean()

qr_train.loc['贵州',  '黔南州']=qr_train.loc['贵州'].mean()
qc_train.loc['贵州',  '黔南州']=qc_train.loc['贵州'].mean()

qr_train.loc['贵州', '黔东南州']=qr_train.loc['贵州'].mean()
qc_train.loc['贵州', '黔东南州']=qc_train.loc['贵州'].mean()

qr_train.loc['贵州', '黔西南州']=qr_train.loc['贵州'].mean()
qc_train.loc['贵州', '黔西南州']=qc_train.loc['贵州'].mean()

qr_train.loc['吉林',  '梅河口']=qr_train.loc['吉林'].mean()
qc_train.loc['吉林',  '梅河口']=qc_train.loc['吉林'].mean()

qc_bili.reset_index(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\P

1.3分割省市,目的是将多个表通过索引连接在一起

In [12]:
areas=[]
for i in qc_bili.iloc[:,0].values:
    if '市' in i:
        w=i.split('市')
        areas.append([w[1][:2],w[0]])
    elif '州' in i:
        z=i.split('州')
        areas.append([z[1][:2],z[0]])

areas_df = pd.DataFrame(areas,columns=['省份','地区'])
areas_df.iloc[94,0] = '贵州'
areas_df.iloc[94,1] = '黔东南州'
areas_df.iloc[11,1] = '恩施州'
areas_df.iloc[61][0] = '黑龙江'

In [16]:
areas_df

,省份,地区
0,湖北,孝感
1,湖北,黄冈
2,湖北,荆州
3,湖北,咸宁
4,湖北,鄂州
...,...,...
95,浙江,台州
96,湖南,怀化
97,辽宁,大连
98,湖南,湘潭


In [17]:
qc_bili.drop(qc_bili.columns[0],axis=1,inplace=True)

In [6]:
qc_bili[['省份','地区']] = areas_df[['省份','地区']]

In [7]:
qc_bili.set_index(['省份','地区'],inplace=True)

In [8]:
qc_bili_nan = qc_bili[qc_bili.isna().values==True].drop_duplicates()

index_nan=qc_bili_nan.index
# 将武汉迁出比例缺失值补为平均值
k=0
for i in index_nan:
    for j in range(len(qc_bili.loc[i])):
        qc_bili.loc[i].iloc[j]=qc_bili_nan.mean(axis=1).values[k]
    k=k+1
        
qc_train.reset_index(inplace=True)
qc_bili.reset_index(inplace=True)
qr_train.reset_index(inplace=True)

qc_bili.set_index(['省份','地区'],inplace=True)
qc_train.rename(columns={'迁徙规模指数（单位）：反映迁入或迁出人口规模，城市间可横向对比':'省份'},inplace=True)
qc_train.set_index(['省份','地区'],inplace=True)
qr_train.rename(columns={'迁徙规模指数（单位）：反映迁入或迁出人口规模，城市间可横向对比':'省份'},inplace=True)

qr_train.set_index(['省份','地区'],inplace=True)

F:\Python\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


1.4将迁入、迁出和比例连接到一起，最终形成训练集

In [10]:
train1 = pd.merge(qr_train,qc_train,right_index=True,left_index=True,how='outer')
train2 = pd.merge(train1,qc_bili,right_index=True,left_index=True,how='outer')

a = {
    '1.1_x' : '1.10迁入指数',
    '1.11_x' : '1.11迁入指数',
    '1.12_x' :  '1.12迁入指数',
    '1.13_x' : '1.13迁入指数',
    '1.14_x' : '1.14迁入指数',
    '1.15_x' : '1.15迁入指数',
    '1.16_x' : '1.16迁入指数',
    
    '1.1_y' : '1.10迁出指数',
    '1.11_y' : '1.11迁出指数',
    '1.12_y' : '1.12迁出指数',
    '1.13_y' : '1.13迁出指数',
    '1.14_y' : '1.14迁出指数',
    '1.15_y' :  '1.15迁出指数',
    '1.16_y' : '1.16迁出指数',
    
    1.1 : '1.10比例',
    1.11 : '1.11比例',
    1.12 : '1.12比例',
    1.13 :  '1.13比例',
    1.14 : '1.14比例',
    1.15 : '1.15比例',
    1.16 : '1.16比例',
}

train2.rename(columns=a,inplace=True)

train2.fillna(0,inplace=True)   # 把比例中武汉人没去的地区补为0

# 2.测试集数据预处理

In [11]:
qr=pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='迁入',index_col=[0,1])
qc=pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='迁出',index_col=[0,1])
qc_bili_test = pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='1.17-1.23武汉迁出比例',index_col=0)

# qr.set_index(['迁徙规模指数（单位）：反映迁入或迁出人口规模，城市间可横向对比','地区'])
qc_test = qc.iloc[:,7:-1]
qr_test = qr.iloc[:,7:-1]

qr_test.fillna(0,inplace=True)
qc_test.fillna(0,inplace=True)

F:\Python\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [12]:
qr_test.loc['江西', '赣江新区']=qr_test.loc['江西'].mean()
qc_test.loc['江西', '赣江新区']=qc_test.loc['江西'].mean()

qr_test.loc['海南','三沙']=qr_test.loc['海南'].mean()
qc_test.loc['海南',   '三沙']=qc_test.loc['海南'].mean()

qr_test.loc['贵州',  '黔南州']=qr_test.loc['贵州'].mean()
qc_test.loc['贵州',  '黔南州']=qc_test.loc['贵州'].mean()

qr_test.loc['贵州', '黔东南州']=qr_test.loc['贵州'].mean()
qc_test.loc['贵州', '黔东南州']=qc_test.loc['贵州'].mean()

qr_test.loc['贵州', '黔西南州']=qr_test.loc['贵州'].mean()
qc_test.loc['贵州', '黔西南州']=qc_test.loc['贵州'].mean()

qr_test.loc['吉林',  '梅河口']=qr_test.loc['吉林'].mean()
qc_test.loc['吉林',  '梅河口']=qc_test.loc['吉林'].mean()

F:\Python\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
F:\Python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
F:\Python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
F:\Python\lib\site-packages\ipykernel_launcher

In [13]:
qc_bili_test.reset_index(inplace=True)

In [14]:
areas=[]
for i in qc_bili_test.iloc[:,0].values:
    if '市' in i:
        w=i.split('市')
        areas.append([w[1][:2],w[0]])
    elif '州' in i:
        z=i.split('州')
        areas.append([z[1][:2],z[0]])

areas_df = pd.DataFrame(areas,columns=['省份','地区'])

In [15]:
areas_df.iloc[12,1] =  '恩施州'
areas_df.iloc[57,0] = '黑龙江'

In [16]:
qc_bili_test.drop(qc_bili_test.columns[0],axis=1,inplace=True)

qc_bili_test[['省份','地区']] = areas_df[['省份','地区']]

qc_bili_test.set_index(['省份','地区'],inplace=True)

qc_bili_nan = qc_bili_test[qc_bili_test.isna().values==True].drop_duplicates()

index_nan=qc_bili_nan.index

k=0
for i in index_nan:
    for j in range(len(qc_bili_test.loc[i])):
        qc_bili_test.loc[i].iloc[j]=qc_bili_nan.mean(axis=1).values[k]
    k=k+1
        
qc_test.reset_index(inplace=True)
qc_bili_test.reset_index(inplace=True)
qr_test.reset_index(inplace=True)

qc_bili_test.set_index(['省份','地区'],inplace=True)

qc_test.rename(columns={'迁徙规模指数（单位）：反映迁入或迁出人口规模，城市间可横向对比':'省份'},inplace=True)

qc_test.set_index(['省份','地区'],inplace=True)

qr_test.rename(columns={'迁徙规模指数（单位）：反映迁入或迁出人口规模，城市间可横向对比':'省份'},inplace=True)

qr_test.set_index(['省份','地区'],inplace=True)


test1 = pd.merge(qr_test,qc_test,right_index=True,left_index=True,how='outer')

test888 = pd.merge(test1,qc_bili_test,right_index=True,left_index=True,how='outer')

a = {
    '1.17_x' : '1.17迁入指数',
    '1.18_x' : '1.18迁入指数',
    '1.19_x' : '1.19迁入指数',
    '1.2_x' : '1.20迁入指数',
    '1.21_x' :  '1.21迁入指数',
    '1.22_x' : '1.22迁入指数',
    '1.23_x' : '1.23迁入指数',
    
    '1.17_y' : '1.17迁出指数',
    '1.18_y' : '1.18迁出指数',
    '1.19_y' : '1.19迁出指数',
    '1.2_y' : '1.20迁出指数',
    '1.21_y' : '1.21迁出指数',
    '1.22_y' : '1.22迁出指数',
    '1.23_y' : '1.23迁出指数',
    
    1.17 : '1.17比例',
    1.18 : '1.18比例',
    1.19 : '1.19比例',
    1.2 : '1.20比例',
    1.21 : '1.21比例',
    1.22 : '1.22比例',
    1.23 : '1.23比例',
}

test888.rename(columns=a,inplace=True)

test888.fillna(0,inplace=True)

F:\Python\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# 3.导出excel

3.1删除冗余数据，并将训练集和测试集分别到出excel文件

In [17]:
ys = pd.read_excel('./AI 2020 04 data01.xlsx',sheet_name='疫情数据（累计）',index_col=[0,1])

ys.fillna(0,inplace=True)

ys['y_训练集'] = ys['2020-01-30']-ys['2020-01-24']

ys['y_测试集'] = ys.iloc[:,-2]-ys.iloc[:,-3]

In [18]:
finaly_train = pd.merge(train2,ys['y_训练集'],right_index=True,left_index=True).reset_index().drop(columns='单位：人（空白为该时间点数据没有记录）').set_index(['省份','地区'])

In [19]:
finaly_test = pd.merge(test888,ys['y_测试集'],right_index=True,left_index=True).reset_index().drop(columns='单位：人（空白为该时间点数据没有记录）').set_index(['省份','地区'])

In [20]:
finaly_train.to_excel(r'./训练集.xlsx')
finaly_test.to_excel(r'./测试集.xlsx')